# Orquestación Secuencial para Workflow de Ingeniería Aeroespacial

## Descripción

En este ejercicio, dominarás el **patrón de orquestación secuencial** de Microsoft Agent Framework para crear un workflow colaborativo. Tres agentes especializados procesarán secuencialmente un problema técnico: extracción de requisitos, evaluación de riesgos y planificación de acciones.

### Arquitectura Secuencial
```
Entrada Técnica
    ↓
[Requirements Extractor]
    ↓
[Risk Assessor]
    ↓
[Action Planner]
    ↓
Plan de Acción Priorizado
```

### Conceptos Clave
- **SequentialBuilder**: Constructor de workflows secuenciales
- **WorkflowOutputEvent**: Captura salidas de cada etapa
- **Contexto persistente**: Cada agente ve el trabajo de los anteriores

### Cargar librerías y variables de entorno

In [ ]:
import asyncio
from dotenv import load_dotenv, find_dotenv
from typing import cast
from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential

load_dotenv(find_dotenv(usecwd=True))

### Definir las Instrucciones de los Agentes

Cada agente tiene un rol específico en el flujo de trabajo: extractor de requisitos, evaluador de riesgos y planificador de acciones.

In [ ]:
extractor_instructions="""
Eres un Ingeniero de Sistemas Aeronáuticos. Extrae de la entrada:
    - requisitos técnicos explícitos e implícitos
    - subsistemas implicados (motores, aviónica, estructura, control de vuelo, hidráulico, combustible)
    - restricciones regulatorias (EASA/FAA), métricas clave y supuestos
Devuelve un resumen técnico claro y objetivo en 1-2 frases.
"""

risk_instructions="""
Eres un Ingeniero de Aeronavegabilidad y Seguridad.
Evalúa riesgos técnicos derivados del resumen: seguridad, cumplimiento (CS-25/Part 25), performance y certificación.
Prioriza la criticidad (Alta/Media/Baja) y señala datos faltantes.
Devuelve 1 frase con la criticidad y 1 con riesgos principales.
"""

planner_instructions="""
Eres un Planificador Técnico Aeroespacial.
Propón la siguiente acción concreta en 1-2 frases: p.ej., análisis FMEA, campaña NDT, simulación CFD/FEA,
emisión de Service Bulletin, prueba en banco, actualización de MM/IPC, coordinación con Supply Chain o MRO.
Incluye responsable sugerido (Equipo Motor/Estructuras/Aviónica/Flight Controls/QA)."""

### Crear y ejecutar la orquestación secuencial

In [ ]:
async def process_technical_case(prompt):
    credential = AzureCliCredential()
    async with AzureAIAgentClient(async_credential=credential) as chat_client:
        extractor = chat_client.create_agent(
            instructions=extractor_instructions,
            name="requirements_extractor",
        )
        print("[1] Requirements extractor created")
        
        risk = chat_client.create_agent(
            instructions=risk_instructions,
            name="risk_assessor",
        )
        print("[2] Risk assessor created")
        
        planner = chat_client.create_agent(
            instructions=planner_instructions,
            name="action_planner",
        )
        print("[3] Action planner created")

        workflow = SequentialBuilder().participants([extractor, risk, planner]).build()

        outputs: list[list[ChatMessage]] = []
        print(f"\nProcessing technical case: {prompt}")
        async for event in workflow.run_stream(f"Technical input: {prompt}"):
            if isinstance(event, WorkflowOutputEvent):
                outputs.append(cast(list[ChatMessage], event.data))

        if outputs:
            print("\nOrchestration results:")
            for i, msg in enumerate(outputs[-1], start=1):
                if (msg.author_name is None):
                    continue
                name = msg.author_name
                print(f"{'=' * 20} {name} {'=' * 20}\n{msg.text}\n")

# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Incremento de vibraciones en fase de ascenso, canal de aviónica reporta fallos intermitentes ARINC 429 en ADAHRS; sin registros de fallo duro. ¿Plan?"
await process_technical_case(user_prompt)

In [ ]:
# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Solicitud de aumento de MTOW requiere refuerzo en sección 36; impacto en performance y certificación CS-25/Part 25. Propón camino de validación."
await process_technical_case(user_prompt)

In [ ]:
# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Solicito trazabilidad digital completa (CoC, FAIR, lotes, heat treatment) integrada con PLM/MES para carcasas de compresor del programa UltraFan."
await process_technical_case(user_prompt)